In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick

## Data Parsing

In [ ]:
def parse_data(file_path):
    df = pd.read_csv(file_path)
    if all(elem in df.columns for elem in ["Date", "High", "Low", "Open", "Close"]):
        df = df[["Date", "High", "Low", "Open", "Close"]]
    else:
        raise Exception("Required columns: {}".format(["Date", "High", "Low", "Open", "Close"]))
        return None
    
    df["High_std"] = df["High"].rolling(3).std()
    df["Low_std"] = df["Low"].rolling(3).std()

    df["High_max"] = df["High_std"].rolling(3).max()
    df["Low_max"] = df["Low_std"].rolling(3).max()
    
    df["MA"] = df["Close"].rolling(3).mean()
    df["PPVI_High"] = df["MA"] + 2 * df["High_max"]
    df["PPVI_Low"] = df["MA"] - 2 * df["Low_max"]
    return df

In [ ]:
BTC = parse_data("../input/cryptocurrencypricehistory/coin_Bitcoin.csv")
BTC.head()

## Visualization

In [ ]:
def plot_ohlc(data):
    fig1, ax1 = plt.subplots(figsize=(12, 3), constrained_layout=True)
    
    for i in data.index:
        ax1.vlines(x=i, ymin=data["Low"][i], ymax=data["High"][i], color="black", linewidth=1)
        if data["Close"][i] > data["Open"][i]:
            ax1.vlines(x=i, ymin=data["Open"][i], ymax=data["Close"][i], color="green", linewidth=3)
        elif data["Close"][i] < data["Open"][i]:
            ax1.vlines(x=i, ymin=data["Close"][i], ymax=data["Open"][i], color="red", linewidth=3)
        elif data["Close"][i] == data["Open"][i]:
            ax1.vlines(x=i, ymin=data["Open"][i], ymax=data["Close"][i]+0.000001, color="black", linewidth=6)
        
    ax1.grid()

plot_ohlc(BTC.tail(100))

In [ ]:
def plot_ohlc_with_indicator(data):
    fig1, ax1 = plt.subplots(figsize=(12, 6), constrained_layout=True)
    
    ax1.plot(data.index, data["PPVI_High"], c="orange")
    ax1.plot(data.index, data["PPVI_Low"], c="orange")
    for i in data.index:
        ax1.vlines(x=i, ymin=data["Low"][i], ymax=data["High"][i], color="black", lw=1)
        if data["Close"][i] > data["Open"][i]:
            ax1.vlines(x=i, ymin=data["Open"][i], ymax=data["Close"][i], color="green", lw=3)
        elif data["Close"][i] < data["Open"][i]:
            ax1.vlines(x=i, ymin=data["Close"][i], ymax=data["Open"][i], color="red", lw=3)
        elif data["Close"][i] == data["Open"][i]:
            ax1.vlines(x=i, ymin=data["Open"][i], ymax=data["Close"][i]+0.000001, color="black", lw=6)
        
    ax1.grid()

plot_ohlc_with_indicator(BTC[1000:1250])

## Back Testing

In [ ]:
def test_trading_strategy(data, holding_period=3):
    holding_period = holding_period
    returns = [1] * holding_period
    position = False
    n_positions = 0
    remaining_days = 0
    entry_price = None
    exit_price = None
    close_prices = data["Close"]
    buy_signal = data["PPVI_Low"]
    sell_signal = data["PPVI_High"]

    for i in range(holding_period, len(close_prices) - holding_period):
        # open position
        if not position:
            # buy signal
            if close_prices[i] < buy_signal[i]:
                position = "buy"
                n_positions += 1
                remaining_days = holding_period
                entry_price = close_prices[i]
            # sell signal
            elif close_prices[i] > sell_signal[i]:
                position = "sell"
                n_positions += 1
                remaining_days = holding_period
                entry_price = close_prices[i]
            returns.append(1)
        # close position
        elif remaining_days == 0:
            exit_price = close_prices[i]
            if position == "buy":
                r = exit_price / entry_price
            elif position == "sell":
                r = entry_price / exit_price
            position = False
            returns.append(r)
        # reduce remaining days
        else:
            remaining_days -= 1
            returns.append(1)
    returns += [1] * holding_period
        
    # Return Statistics
    returns = np.array(returns)
    equity = np.cumprod(returns)
    quality = np.sum(returns > 1) / n_positions

    fig, ax = plt.subplots(figsize=(12, 6), constrained_layout=True)

    ax.plot(equity, color="blue")
    ax.set_title("Equity Return / Signal quality: {:.2%}".format(quality))
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0, decimals=2))
    ax.grid()
        
    return returns, equity, quality

BTC_stat = test_trading_strategy(BTC, holding_period=2)